# Animal Detection

## Table of Contents

1. Overview
4. Dependencies
5. Data
6. Model
7. Model Evaluation
9. Exercises

## 1. Overview

Integrated Gradients is a technique used in image analysis of animals to understand 
which pixels contribute most to a model's prediction. It calculates pixel-wise attributions, 
revealing the importance of each part of an image in the classification decision. This 
helps researchers and conservationists:

1. **Identify Key Features**: By highlighting specific regions, Integrated Gradients 
can pinpoint the visual characteristics that influence a model's classification, such 
as unique markings or features on animals.

2. **Explain Model Decisions**: It provides interpretable explanations for why an AI 
system classified an image in a particular way, aiding in understanding the model's 
decision-making process.

3. **Detect Biases**: Integrated Gradients can help reveal if a model relies on 
biased or irrelevant features, which is crucial for mitigating biases and ensuring 
fairness in wildlife analysis.

4. **Improve Models**: Researchers can use these insights to refine models, enhance 
classification accuracy, and contribute to better wildlife conservation efforts.

5. **Educate and Raise Awareness**: Transparent explanations generated by Integrated 
Gradients can be used to educate the public and raise awareness about animal identification 
and conservation challenges, promoting broader engagement in conservation efforts.

## 2. Dependencies

Here are the packages we will be using in this notebook.

- `matplotlib`
- `numpy`
- `alibi`
- `datasets`
- `tensorflow==2.8`

Please note, due to some mismatch between the latest tensorflow image and other tools, 
you will need to pin the version of tensorflow.

In [ ]:
!pip install datasets matplotlib 'alibi[tensorflow]' numpy rich tensorflow==2.8

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from alibi.explainers import IntegratedGradients
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from alibi.datasets import load_cats
from alibi.utils import visualize_image_attr
print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly())

## 3. Data

The data here can be your own, personally curated one. We will first load 4 samples of cats, 
then move on to the beautiful luna, and finish with different examples.

In [ ]:
image_shape = (224, 224, 3)

In [ ]:
data, labels = load_cats(target_size=image_shape[:2], return_X_y=True)

In [ ]:
print(f'Images shape: {data.shape}')

In [ ]:
data = (data / 255).astype('float32')

In [ ]:
i = 1
plt.imshow(data[i]);

In [ ]:
img = Image.open('data/images/luna_resized.png')

In [ ]:
img_array = np.asarray(img)
img = (img_array / 255).astype('float32')

In [ ]:
(img[None]).shape

## 4. Model

ResNet50 is a convolutional neural network architecture that is 50 layers deep and 
is commonly used for image classification tasks. Here are two ways to think of ResNets:

For practitioners:

ResNet50 is a residual neural network first introduced in 2015. It consists of 5 
stages stacked together, with each stage having a convolution layer followed by 
identity mappings that skip over a few convolution layers. This "skip connection" 
structure allows information to shortcut across layers, avoiding the vanishing 
gradient problem when training very deep networks. After the convolutions, there 
is an average pooling layer and fully connected layer for the output. The 50 in 
ResNet50 refers to it having 50 weight layers. ResNet50 achieved state-of-the-art 
accuracy on ImageNet classification while being easier to optimize than previous 
deep models. It is widely used as a powerful pretrained feature extractor for 
computer vision tasks.

For non-practitioners: 

ResNet50 is like a very deep maze (50 layers) that images can go through to be 
classified into categories like dogs, cats, cars etc. Going through such a deep 
maze makes it hard for information to flow from the beginning to the end. To solve 
this, ResNet50 adds shortcut tunnels between some of the layers. So some information 
can skip ahead instead of getting lost. This allows ResNet50 to be trained very 
accurately on huge image datasets like ImageNet. The whole network acts like a smart 
feature extractor that can pick out patterns useful for identifying objects. This 
knowledge can then be transferred to classify new images by connecting ResNet50 to 
a simpler network. The shortcut design enables ResNet50 to successfully train and 
extract powerful features from images despite being super deep.

In [ ]:
model = ResNet50V2(weights='imagenet')

## 5. Model Evaluation

Integrated Gradients is a method to explain individual predictions for deep neural networks by attributing importance to input features.

Imagine a neural network that classifies images of animals. We want to explain why it predicted "bird" for a particular photo. 

1. Take the input image and a baseline image (e.g. a solid gray image). 
2.  Interpolate between the baseline and input image in small steps. So we get images 
slowly going from gray to the original.
3. At each step, pass the interpolated image into the network to get a prediction. 
4. Calculate the gradients of the prediction with respect to the input pixels at each 
step. The gradients indicate how sensitive the prediction is to changes in each pixel.
5. Integrate the gradients across all the steps. This gives importance scores for each pixel.

Pixels with high integrated gradients contributed significantly to pushing the network from an uninformative baseline to predicting "bird". These pixels are most important for the decision.

An analogy is explaining why a cake tastes sweet. We take small steps adding ingredients to a baseline of an empty bowl:

1) Interpolate between empty bowl and final cake batter 
2) Taste each step, measure change in sweetness
3) Integrate to get importance of each ingredient to sweetness

This reveals sugar as highly important, while flour is less so.

In [ ]:
n_steps = 20
method = "gausslegendre"
internal_batch_size = 20

ig = IntegratedGradients(model, n_steps=n_steps, method=method, internal_batch_size=internal_batch_size)

In [ ]:
img[None].shape

In [ ]:
instance = np.expand_dims(img, axis=0)

In [ ]:
predictions = model(instance).numpy().argmax(axis=1)

In [ ]:
explanation = ig.explain(
    instance, baselines=None, target=predictions
)

In [ ]:
# Metadata from the explanation object
explanation.meta

In [ ]:
# Data fields from the explanation object
explanation.data.keys()

In [ ]:
# Get attributions values from the explanation object
attrs = explanation.attributions[0]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
visualize_image_attr(
    attr=None, original_image=img, method='original_image',
    title='Original Image', plt_fig_axis=(fig, ax[0]), use_pyplot=False
);

visualize_image_attr(
    attr=attrs.squeeze(), original_image=img, method='blended_heat_map',
    sign='all', show_colorbar=True, title='Overlaid Attributions',
    plt_fig_axis=(fig, ax[1]), use_pyplot=True
);

In [ ]:
baselines = np.random.random_sample(instance.shape)

In [ ]:
explanation = ig.explain(
    instance, baselines=baselines, target=predictions
)

In [ ]:
attrs = explanation.attributions[0]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

visualize_image_attr(
    attr=None, original_image=data[i], method='original_image',
    title='Original Image', plt_fig_axis=(fig, ax[0]), use_pyplot=False
);

visualize_image_attr(
    attr=attrs.squeeze(), original_image=data[i], method='blended_heat_map',
    sign='all', show_colorbar=True, title='Overlaid Attributions',
    plt_fig_axis=(fig, ax[1]), use_pyplot=True
);

## 6. Exercises

Find pictures of things that you like and see try to evaluate the following.
- which pixel are influencing the prediction the most?
- what would happen if I change any of these pixes by a little bit or a lot? Will the 
model still predict the correct class?